# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
data = pd.read_csv('customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [2]:
df = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [3]:
df1 = df.pivot_table(values='Quantity', 
                     columns='CustomerID', 
                     index='ProductName', 
                     aggfunc='sum').fillna(0)

df1.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [4]:
df2 = df1.T.head()
df2

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [5]:
pcp = df1.div(df1.sum(axis=1), axis=0)
cpp = df2.div(df2.sum(axis=1), axis=0)


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df2, 'euclidean'))),
                         index=df2.index, columns=df2.index)

distances

CustomerID,33,200,264,356,412
CustomerID,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634
200,0.077421,1.000000,0.078448,0.076435,0.073693
264,0.087047,0.078448,1.000000,0.080070,0.081800
356,0.081800,0.076435,0.080070,1.000000,0.076435
412,0.080634,0.073693,0.081800,0.076435,1.000000


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [7]:
cust_sim = list(distances[33].sort_values(ascending=False)[1:].head().index)
cust_sim

[264, 356, 412, 200]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [8]:
sim_cust_prod = df[df['CustomerID'].isin(cust_sim)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Mayonnaise - Individual Pkg,4
1,Remy Red,4
2,Lamb - Ground,4
3,"Crab - Dungeness, Whole",3
4,Potatoes - Idaho 100 Count,3


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [9]:
merged = pd.merge(ranked_products, pd.DataFrame(df2.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Mayonnaise - Individual Pkg,4,0.0
1,Remy Red,4,0.0
2,Lamb - Ground,4,0.0
3,"Crab - Dungeness, Whole",3,0.0
5,"Spoon - Soup, Plastic",3,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [10]:
prod_distances = pd.DataFrame(1/(1 + squareform(pdist(df2, 'euclidean'))),
                         index=df2.index, columns=df2.index)

prod_distances

CustomerID,33,200,264,356,412
CustomerID,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634
200,0.077421,1.000000,0.078448,0.076435,0.073693
264,0.087047,0.078448,1.000000,0.080070,0.081800
356,0.081800,0.076435,0.080070,1.000000,0.076435
412,0.080634,0.073693,0.081800,0.076435,1.000000


In [26]:
def get_recommendation(ID):
    

    
    top_most_similar = list(afinidad_df[ID].sort_values(ascending=False)[1:].head().index)
    
    top_most_similar_df = customer_prod[customer_prod.CustomerID.isin(top_most_similar)]
    
    grouped = similar_customer_prod.groupby('ProductName').agg({'Quantity':'sum'}) 
    
    ranked_prod = grouped.sort_values('Quantity',ascending=False).reset_index()
    
    
    merged = pd.merge(ranked_prod, pd.DataFrame(customer_prod_pivot_t.T[ID]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    
    return merged[merged['Purchased']==0].head()['ProductName'].tolist()



### Step 2: Get the products purchased for a specific customer of your choice.

In [27]:
rand_cust = distances.sample(1).index[0]
rand_cust_pp = df[df.CustomerID == rand_cust]
rand_cust_pp.head()

,CustomerID,ProductName,Quantity
254,412,Apricots - Dried,1
255,412,Banana - Leaves,1
256,412,Bananas,1
257,412,Beans - Wax,1
258,412,Beef - Rib Eye Aaa,1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [39]:
sim_lst = [list(df[x].sort_values(ascending=False).head(6).index)[1:] for x in rand_cust_pp['ProductName']]
#sim_lst = [it for item in sim_lst for it in item if it not in rand_cust_pp['ProductName']]

NameError: name 'sim_lst' is not defined

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.